<a href="https://colab.research.google.com/github/Maxxx-VS/The-Founder/blob/master/38_3_Transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Установка необходимых пакетов
!apt install --allow-change-held-packages libcudnn8=8.1.0.77-1+cuda11.2
!pip uninstall -y -q tensorflow keras tensorflow-estimator tensorflow-text
!pip install protobuf~=3.20.3
!pip install -q tensorflow_datasets
!pip install -q -U tensorflow-text tensorflow

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Package libcudnn8 is not available, but is referred to by another package.
This may mean that the package is missing, has been obsoleted, or
is only available from another source

E: Version '8.1.0.77-1+cuda11.2' for 'libcudnn8' was not found
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 2.7 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.25.6
    Uninstalling protobuf-4.25.6:
      Successfully uninstalled protobuf-4.25.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
grpcio-status 1.62.3 requires protobuf>=4.21.6, but you have protobuf 3.20.3 which is incompatible.
tensorflow-metadata 1.16.1 requires protobuf<6.0.0dev,>=4.25.2; python_version >= "3.11", but you have protobuf 3.20.3 which is incompatible.


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.7/319.7 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.4/615.4 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 58.9 MB/s eta 0:00:00


In [2]:
# Импорт необходимых библиотек
import logging
import time
import numpy as np
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds
import tensorflow as tf
import tensorflow_text as text
import re
import pathlib
from tensorflow_text.tools.wordpiece_vocab import bert_vocab_from_dataset as bert_vocab
import warnings
warnings.filterwarnings("ignore")

In [3]:
ds = tfds.load('web_questions', split='train', shuffle_files=True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/3778 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/web_questions/incomplete.KRH68Z_1.0.0/web_questions-train.tfrecord*...:   …

Generating test examples...:   0%|          | 0/2032 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/web_questions/incomplete.KRH68Z_1.0.0/web_questions-test.tfrecord*...:   0…

Dataset web_questions downloaded and prepared to /root/tensorflow_datasets/web_questions/1.0.0. Subsequent calls will reuse this data.


In [4]:
ds

<_PrefetchDataset element_spec={'answers': TensorSpec(shape=(None,), dtype=tf.string, name=None), 'question': TensorSpec(shape=(), dtype=tf.string, name=None), 'url': TensorSpec(shape=(), dtype=tf.string, name=None)}>

In [4]:
# examples, metadata = tfds.load('web_questions', with_info=True, as_supervised=False)
# train_examples, val_examples = examples['train'], examples['test']

In [5]:
# print('Размер обучающей выборки: ', len(train_examples))
# print('Размер валидационной выборки: ', len(val_examples))

Размер обучающей выборки:  3778
Размер валидационной выборки:  2032


In [23]:
# print(train_examples)
# print(val_examples)

<_PrefetchDataset element_spec={'answers': TensorSpec(shape=(None,), dtype=tf.string, name=None), 'question': TensorSpec(shape=(), dtype=tf.string, name=None), 'url': TensorSpec(shape=(), dtype=tf.string, name=None)}>
<_PrefetchDataset element_spec={'answers': TensorSpec(shape=(None,), dtype=tf.string, name=None), 'question': TensorSpec(shape=(), dtype=tf.string, name=None), 'url': TensorSpec(shape=(), dtype=tf.string, name=None)}>


In [5]:
i = 1
for example in ds:
    if i < 4:
        print(f"Пример из обучающей выборки №{i}:")
        print(*example['answers'].numpy())
        print(example['question'].numpy())
        print(example['url'].numpy(), "\n")
        i += 1

Пример из обучающей выборки №1:
b'Joe Flacco'
b'who is the ravens quarterback 2012?'
b'http://www.freebase.com/view/en/baltimore_ravens' 

Пример из обучающей выборки №2:
b'Whig Party'
b'what political party was henry clay?'
b'http://www.freebase.com/view/en/henry_clay' 

Пример из обучающей выборки №3:
b'UTC+07:00'
b'where to stay in thailand koh samui?'
b'http://www.freebase.com/view/en/ko_samui' 



In [ ]:
bert_tokenizer_params=dict(lower_case=True)


In [ ]:
pt_tokenizer = text.BertTokenizer('pt_vocab.txt', **bert_tokenizer_params)
en_tokenizer = text.BertTokenizer('en_vocab.txt', **bert_tokenizer_params)

In [ ]:
# Токенизируем примеры и получаем тензор формы (batch, word, word-piece)
token_batch = en_tokenizer.tokenize(en_examples)
# Объединяем оси word и word-piece и получаем тензор формы (batch, tokens)
token_batch = token_batch.merge_dims(-2,-1)

print('Токенизируем отобранные строки:')
for ex in token_batch.to_list():
  print(ex)